## Using the DFT Hamiltonian to learn activation barriers for chemical reactions

DFT computes the energies by solving the generalised eigenvalue problem:

$$
\boldsymbol{H(R)}\psi_i = \epsilon_i \boldsymbol{S(R)}\psi_i
$$

where $\boldsymbol{H(R)}$ is the Hamiltonian Matrix, $\boldsymbol{S(R)}$ is the overlap matrix, $\epsilon_i$ is the computed eigenvalue for state i, with an eigenvector $\psi_i$.

$\boldsymbol{H(R)}$ and $\boldsymbol{S(R)}$ change with rotation, which means we have to process them appropriately prior to using them as features in our NN models.

In [1]:
import numpy as np
import plotly.express as px
from instance_mongodb import instance_mongodb_sei

from pymatgen.core.structure import Molecule

import torch

from monty.serialization import loadfn, dumpfn

from e3nn import o3

from utils import rotate_three_dimensions, subdiagonalize_matrix

In [2]:
def add_bounds_to_figure(fig):

    fig.add_shape(
        type="line",
        x0=-0.5,
        y0=4.5,
        x1=6.5,
        y1=4.5,
        line=dict(color="black", width=2),
    )
    fig.add_shape(
        type="line",
        x0=-0.5,
        y0=5.5,
        x1=6.5,
        y1=5.5,
        line=dict(color="black", width=2),
    )
    fig.add_shape(
        type="line",
        x0=-0.5,
        y0=-0.5,
        x1=-0.5,
        y1=6.5,
        line=dict(color="black", width=2),
    )
    fig.add_shape(
        type="line",
        x0=4.5,
        y0=-0.5,
        x1=4.5,
        y1=6.5,
        line=dict(color="black", width=2),
    )

    fig.update_xaxes(
        ticktext=[ "O1s", "O2s", "O2p", "O2p", "O2p", "H1s", "H1s",],
        tickvals=np.arange(11),
    )
    fig.update_yaxes(
        ticktext=["O1s", "O2s", "O2p", "O2p", "O2p", "H1s", "H1s"],
        tickvals=np.arange(11),
    )

    return fig

In [3]:
db = instance_mongodb_sei(project="mlts")
collection = db.rotated_waters_dataset

hamiltonians = []
angles = []
structures = []
overlap_matrices = []
coefficients = []
eigenvalues = []

for doc in collection.find({}).limit(10):
    hamiltonians.append(doc["fock_matrices"][0])
    overlap_matrices.append(doc["overlap_matrices"][0])
    structure = Molecule.from_dict(doc["structures"][0])
    structures.append(structure)
    coefficients.append(doc["coeff_matrices"][0])
    angles.append(doc["angles"])
    eigenvalues.append(doc["eigenvalues"][0])

hamiltonians = np.array(hamiltonians)
overlap_matrices = np.array(overlap_matrices)
coefficients = np.array(coefficients)
angles = np.array(angles)
eigenvalues = np.array(eigenvalues)
eigenvalues = eigenvalues[..., np.newaxis, :]

In [4]:
fig = px.imshow(
    hamiltonians[:, 1, ...], animation_frame=0, labels=dict(x="Basis", y="Basis", color="Value"),
    range_color=[-1, 1],
)
add_bounds_to_figure(fig)
fig.update_layout(title_text='DFT computed Hamiltonian for a rotated water molecule', title_x=0.5)
fig.show()

In [5]:
fig = px.imshow(
    overlap_matrices[:, 1, ...], animation_frame=0, labels=dict(x="Basis", y="Basis", color="Value"),
    range_color=[0, 1],
)
add_bounds_to_figure(fig)
fig.update_layout(title_text='DFT computed overlap matrix for a rotated water molecule', title_x=0.5)
fig.show()


In [6]:
fig = px.imshow(
    eigenvalues[:,0,...], animation_frame=0, labels=dict(x="Basis", y="Eigenvalues", color="Value"),
)
fig.update_layout(title_text='DFT computed eigenvalues for a rotated water molecule', title_x=0.5)
fig.update_yaxes(tickmode='array', tickvals=np.arange(11))
fig.update_xaxes(tickmode='array', tickvals=np.arange(11))
fig.show()

In [7]:
fig = px.imshow(
    coefficients[:, 0, ...], animation_frame=0, labels=dict(x="Basis", y="Basis", color="Value"),
)
add_bounds_to_figure(fig)
fig.update_layout(title_text='DFT computed coefficient matrix for a rotated water molecule', title_x=0.5)
# Remove tick labels from the yaxis
# Change y-label to be `eigenvalues index`
fig.update_xaxes(title_text="Eigenvalues index")
# Change the x-axis labels to be the integral labels
fig.update_xaxes(
    ticktext=np.arange(8),
    tickvals=np.arange(8),
)
fig.show()


### Change coefficient matrix to orthonormal basis

- For each structure and spin, $\sigma$ find the eigenvalues and eigenvectors of the overlap matrix as $\mathbf{S}\Lambda = \lambda \Lambda$
- Create an orthogonalization matrix, $\mathbf{S}^{-1/2} = \lambda \Lambda^{-1/2} \lambda$
- Orthogonalize the Fock matrix as $\mathbf{S}^{-1/2, T} F \mathbf{S}^{-1/2}$
- Find the eigenvalues and eigenvectors of the orgonalized Hamiltonian. These eigenvectors are the coefficient matrix in the orthonormal basis. Hence, the sum of the square of these elements for each row and column is 1.

In [12]:
specific_coeff = coefficients[0, 0, ...]
specific_overlap = overlap_matrices[0, 0, ...]
specific_hamiltonian = hamiltonians[0, 0, ...]
specific_eigenvalues = eigenvalues[0, 0, ...]
print("Original eigenvalues: ", specific_eigenvalues)


eigenval_overlap, eigenvec_overlap = np.linalg.eigh(specific_overlap) 
orthogonalisation_matrix = np.dot(eigenvec_overlap, np.dot(np.diag(1 / np.sqrt(eigenval_overlap)), eigenvec_overlap.T))

orthogonalised_hamiltonian = np.dot(orthogonalisation_matrix.T, np.dot(specific_hamiltonian, orthogonalisation_matrix))
eigenval_hamiltonian, eigenvec_hamiltonian = np.linalg.eigh(orthogonalised_hamiltonian)
sum_of_squares = np.sum(eigenvec_hamiltonian ** 2, axis=-1)
print("Sum of squares of the eigenvectors: ", sum_of_squares)
print("Eigenvalues of orthogonlised hamiltonian: ", eigenval_hamiltonian)

# Convert back to the original basis
orig_eigenvec = np.dot(orthogonalisation_matrix, eigenvec_hamiltonian)
print("Sum of squares of the original eigenvectors: ", np.sum(orig_eigenvec ** 2, axis=-1))
print("Sum of squares of the original eigenvectors: ", np.sum(specific_coeff ** 2, axis=-1))

eigenvec_hamiltonian = np.abs(eigenvec_hamiltonian)
orig_eigenvec = np.abs(orig_eigenvec)
specific_coeff = np.abs(specific_coeff)

to_plot_eigenvec = [ eigenvec_hamiltonian, orig_eigenvec, specific_coeff]
to_plot_eigenvec = np.array(to_plot_eigenvec)
fig = px.imshow(
    to_plot_eigenvec, animation_frame=0, labels=dict(x="Basis", y="Basis", color="Value"),
    # range_color=[0, 2]
)
add_bounds_to_figure(fig)
fig.show()

Original eigenvalues:  [[-18.8369492  -0.9309247  -0.4384445  -0.2325507  -0.1439588   0.3462303
    0.4589058]]
Sum of squares of the eigenvectors:  [1. 1. 1. 1. 1. 1. 1.]
Eigenvalues of orthogonlised hamiltonian:  [-18.83694956  -0.93092466  -0.43844446  -0.2325507   -0.1439588
   0.34623053   0.45890586]
Sum of squares of the original eigenvectors:  [1.07035998 1.73438821 1.23082637 1.26929127 1.00075063 1.61243407
 1.61243377]
Sum of squares of the original eigenvectors:  [1.07035992 1.73438801 1.23082644 1.26929126 1.0007512  1.61243373
 1.61243358]


In order to determine _how_ the Hamiltonian changes as a function of rotation $\left (\alpha, \beta, \gamma \right)$, we create the Wigner-D matrix.

$$
\boldsymbol{H}\left( \alpha, \beta, \gamma \right) = D\left(\alpha, \beta, \gamma\right) \boldsymbol{H} D\left(\alpha, \beta, \gamma\right)^T
$$

In [13]:
irreps_hamiltonian = o3.Irreps("1x0e+1x0e+1x1o+1x0e+1x0e")

D_matrices = []
for idx, angle in enumerate(angles):
    alpha, beta, gamma = angle

    rotation_matrix = rotate_three_dimensions(alpha, beta, gamma)
    rotation_matrix = torch.tensor(rotation_matrix)
    if idx == 0:
        rotation_matrix_0 = rotation_matrix

    # Reference the rotation matrix to the first one
    rotation_matrix = rotation_matrix @ rotation_matrix_0.T

    D_matrix = irreps_hamiltonian.D_from_matrix(rotation_matrix)
    D_matrices.append(D_matrix)

D_matrices = torch.stack(D_matrices)
# Convert to numpy array
D_matrices = D_matrices.detach().numpy()

fig = px.imshow(
    D_matrices, animation_frame=0, labels=dict(x="Basis", y="Basis", color="Value")
)

fig.update_layout(title_text='Wigner Matrix (D)', title_x=0.5)
add_bounds_to_figure(fig)
fig.show()

In [14]:
# Referencing all the angles to the first angle, compute the Hamiltonian in the rotated basis
hamiltonians_rotated = []
for i in range(len(angles)):
    hamiltonian_rotated = D_matrices[i] @ hamiltonians[0, ..., :, :] @ D_matrices[i].T
    hamiltonians_rotated.append(hamiltonian_rotated)

hamiltonians_rotated = np.array(hamiltonians_rotated)
hamiltonians_rotated_diff = hamiltonians_rotated - hamiltonians

fig = px.imshow(
    hamiltonians_rotated_diff[:, 0, ...],
    animation_frame=0,
    labels=dict(x="Basis", y="Basis", color="Value"),
    range_color=[-1, 1],
)

add_bounds_to_figure(fig)

fig.update_layout(title_text='Difference between computed and rotated H', title_x=0.5)

fig.show()

In [15]:
# Check the rotation of the overlap matrix
overlap_matrices_rotated = []
for i in range(len(angles)):
    overlap_matrix_rotated = D_matrices[i] @ overlap_matrices[0, ..., :, :] @ D_matrices[i].T
    overlap_matrices_rotated.append(overlap_matrix_rotated)

overlap_matrices_rotated = np.array(overlap_matrices_rotated)
overlap_matrices_rotated_diff = overlap_matrices_rotated - overlap_matrices

fig = px.imshow(
    overlap_matrices_rotated_diff[:, 0, ...],
    animation_frame=0,
    labels=dict(x="Basis", y="Basis", color="Value"),
    range_color=[-1, 1],
)
add_bounds_to_figure(fig)
fig.show()

In [30]:
# Check the rotation of the coefficient matrix
coeff_chosen_idx = 2
all_data = []
for i in range(len(angles)):
    init_coeff = coefficients[0, 0, :, coeff_chosen_idx] 
    calc_coeff = coefficients[i, 0, :, coeff_chosen_idx]
    coeff_rotated =  init_coeff @ D_matrices[i].T 
    _coefficients_rotated_diff = coeff_rotated - calc_coeff
    _coefficients_rotated_add = coeff_rotated + calc_coeff
    _max_diff = np.max(np.abs(_coefficients_rotated_diff))
    _max_add = np.max(np.abs(_coefficients_rotated_add))
    print(f"Max diff: {_max_diff}, max add: {_max_add}")
    coefficients_rotated_diff = _coefficients_rotated_diff if _max_diff < _max_add else _coefficients_rotated_add 
    _all_data = np.stack([calc_coeff, coeff_rotated, coefficients_rotated_diff])
    all_data.append(_all_data)

all_data = np.array(all_data)

fig = px.imshow(
    all_data,
    animation_frame=0,
    labels=dict(x="Eigenvalue index", color="Value"),
    # range_color=[0, 1],
    title="Rotation of the coefficient matrix"
)
fig.update_yaxes(tickmode='array', tickvals=np.arange(3), ticktext=['computed', 'rotated', 'difference'])
fig.show()


Max diff: 1.4129180078356618e-07, max add: 0.952577
Max diff: 5.703780525112556e-06, max add: 1.043850539748082
Max diff: 0.9937485666434154, max add: 3.3666434153811764e-06
Max diff: 0.9895615871287844, max add: 7.983362481478462e-06
Max diff: 1.0078361090207886, max add: 2.7090207884716833e-06
Max diff: 5.385900782389763e-06, max add: 0.8897774
Max diff: 1.0619595782012698, max add: 3.178201269826708e-06
Max diff: 1.211419212928314, max add: 3.6129283138564006e-06
Max diff: 1.1106516292601176, max add: 7.814412869483478e-06
Max diff: 3.803918723177535e-06, max add: 0.9724856595922609
